## Sample notebook with examples to run rte-rrtmgp using python

### Loading the data

In [1]:
import xarray as xr
import sys
import numpy as np

rte_rrtmgp_dir = "/Users/josue/Documents/makepath/rte-python/rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

rfmip = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)

lw_gas_coeffs = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-lw-g256.nc")

rfmip = rfmip.sel(expt=0) # only one experiment
kdist = lw_gas_coeffs

flxdn_file = xr.load_dataset(
    f"{clear_sky_example_files}/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
)
flxup_file = xr.load_dataset(
    f"{clear_sky_example_files}/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
)


pres_layers = rfmip["pres_layer"]["layer"]
top_at_1 = pres_layers[0] < pres_layers[-1]

# RRTMGP won't run with pressure less than its minimum. so we add a small value to the minimum pressure
press_min = rfmip["pres_level"].min()
min_index = rfmip["pres_level"].argmin()
rfmip["pres_level"][min_index] = press_min + sys.float_info.epsilon

kdist_gas_names = [n.decode().strip() for n in kdist["gas_names"].values]

rfmip_vars = list(rfmip.keys())

gas_names = {n: n+"_GM" for n in kdist_gas_names if n + "_GM" in rfmip_vars}


# There is more than one config, need to work on setting the conditions for that
gas_names.update({
    "co": "carbon_monoxide_GM",
    "ch4": "methane_GM",
    "o2": "oxygen_GM",
    "n2o": "nitrous_oxide_GM",
    "n2": "nitrogen_GM",
    "co2": "carbon_dioxide_GM",
    "ccl4": "carbon_tetrachloride_GM",
    "cfc22": "hcfc22_GM",
    "h2o": "water_vapor",
    "o3": "ozone",
})

In [2]:
# The compu

# k_dist%gas_optics(p_lay(:,:,b), &
#                 p_lev(:,:,b),       &
#                 t_lay(:,:,b),       &
#                 sfc_t(:  ,b),       &
#                 gas_conc_array(b),  &
#                 optical_props,      &
#                 source,             &
#                 tlev = t_lev(:,:,b))

ncol = len(rfmip["site"])
nlay = len(rfmip["layer"])
nbnd = len(kdist["bnd"])
ngpt = len(kdist["gpt"])
neta = len(kdist["mixing_fraction"])
npres = len(kdist["pressure"])
ntemp = len(kdist["temperature"])

sfc_emis = rfmip["surface_emissivity"].values
tsfc = rfmip["surface_temperature"].values
sfc_lay = nlay if top_at_1 else 1 
press_ref_log = np.log(kdist["press_ref"]).values
temp_ref = kdist["temp_ref"].values

tlay = rfmip["temp_layer"].values
tlev = rfmip["temp_level"].values
play = rfmip["pres_layer"].values
plev = rfmip["pres_level"].values

nPlanckTemp = len(kdist["temperature_Planck"])
pfracin = kdist["plank_fraction"].values.transpose(0, 2, 1, 3)
temp_ref_min = kdist["temp_ref"].values.min()
temp_ref_max = kdist["temp_ref"].values.max()
totplnk = kdist["totplnk"].values.T # (nPlanckTemp,nbnd)
totplnk_delta = (temp_ref_max-temp_ref_min) / (nPlanckTemp-1)

band_ranges = [np.arange(s, e+1) for s, e in kdist["bnd_limits_gpt"]]
gpoint_bands = np.concatenate(band_ranges)
band_lims_gpt = kdist["bnd_limits_gpt"].values.T

# build the gas array
# TODO: need to add dry air
col_gas = []
for gas in gas_names.values():
    gas_values = rfmip[gas].values
    if gas_values.ndim == 0:
        gas_values = np.full((ncol, nlay), gas_values) # expand the gas to all columns and layers
    col_gas.append(gas_values)
col_gas = np.stack(col_gas, axis=-1)

# start on 1 as we ignore dry air that is 0
vmr_idx = [i for i, g in enumerate(kdist_gas_names, 1) if g in gas_names]

# dry air is missing
vmr_ref = kdist["vmr_ref"].sel(absorber_ext=vmr_idx)

In [3]:
key_species = kdist["key_species"].values
tot_flav = len(kdist["bnd"]) * len(kdist["atmos_layer"])
npairs = len(kdist["pair"])

all_flav = np.reshape(key_species, (tot_flav, npairs))
# If all elements in key_species_pair are 0, set rewrite_key_species_pair to [2, 2]
# (0,0) becomes (2,2) -- because absorption coefficients for these g-points will be 0.
all_flav[np.all(all_flav == [0, 0], axis=1)] = [2, 2]

flavors = np.unique(all_flav, axis=0).T
nflav = len(flavors[0])

band_ranges = [[i]*(r.values[1]-r.values[0]+1) for i, r in enumerate(kdist["bnd_limits_gpt"], 1)]
gpoint_bands = np.concatenate(band_ranges)

key_species_rep = key_species.copy()
key_species_rep[np.all(key_species_rep==[0, 0], axis=2)] = [2, 2]

flist = flavors.T.tolist()
def key_species_pair2flavor(key_species_pair):
    return flist.index(key_species_pair.tolist()) + 1

flavors_bands = np.apply_along_axis(key_species_pair2flavor, 2, key_species_rep).tolist()
gpoint_flavor = np.array([flavors_bands[gp-1] for gp in gpoint_bands]).T

### Gas optics

The gas optics are calculated using the `gas_optics` for kdist that uses the `source` function from the frontend.

In [4]:
from pyrte_rrtmgp.pyrte_rrtmgp import rrtmgp_interpolation

# outputs
jtemp = np.ndarray([ncol, nlay], dtype=np.int32)
fmajor = np.ndarray([2, 2, 2, ncol, nlay, nflav], dtype=np.float64)
fminor = np.ndarray([2, 2, ncol, nlay, nflav], dtype=np.float64)
col_mix = np.ndarray([2, ncol, nlay, nflav], dtype=np.float64)
tropo = np.ndarray([ncol, nlay], dtype=np.int32)
jeta = np.ndarray([2, ncol, nlay, nflav], dtype=np.int32)
jpress = np.ndarray([ncol, nlay], dtype=np.int32)

interp_kwargs = dict(
    ncol = ncol,
    nlay = nlay,
    ngas = len(gas_names),
    nflav = nflav,
    neta = neta,
    npres = npres,
    ntemp = ntemp,
    flavor = flavors,
    press_ref_log = press_ref_log,
    temp_ref = kdist["temp_ref"].values,
    press_ref_log_delta = round((press_ref_log.min() - press_ref_log.max()) / (len(press_ref_log) - 1), 9),
    temp_ref_min = temp_ref.min(),
    temp_ref_delta = (temp_ref.max() - temp_ref.min()) / (len(temp_ref) - 1),
    press_ref_trop_log = kdist["press_ref_trop"].values.item(),
    vmr_ref = vmr_ref.values.transpose(2, 1, 0),
    play = play,
    tlay = tlay,
    col_gas = col_gas, # uses ngas dim(ncol,nlay,0:ngas)
    jtemp = jtemp,
    fmajor = fmajor,
    fminor = fminor,
    col_mix = col_mix,
    tropo = tropo,
    jeta = jeta,
    jpress = jpress
)

rrtmgp_interpolation(*list(interp_kwargs.values()))

In [5]:
from pyrte_rrtmgp.pyrte_rrtmgp import rrtmgp_compute_Planck_source

# outputs
sfc_src = np.ndarray([ncol, ngpt], dtype=np.float64)
lay_src = np.ndarray([ncol, nlay, ngpt], dtype=np.float64)
lev_src = np.ndarray([ncol, nlay+1, ngpt], dtype=np.float64)
sfc_src_jac = np.ndarray([ncol, ngpt], dtype=np.float64)


planck_kwargs = dict(
        ncol = ncol,
        nlay = nlay,
        nbnd = nbnd,
        ngpt = ngpt,
        nflav = nflav,
        neta = neta,
        npres = npres,
        ntemp = ntemp,
        nPlanckTemp = nPlanckTemp,
        tlay = tlay,
        tlev = tlev,
        tsfc = tsfc,
        sfc_lay = sfc_lay,
        fmajor = fmajor,
        jeta = jeta,
        tropo = tropo!=0,
        jtemp = jtemp,
        jpress = jpress,
        gpoint_bands = gpoint_bands, # (ngpt)
        band_lims_gpt = band_lims_gpt, # (2, nbnd)
        pfracin = pfracin,
        temp_ref_min = temp_ref_min,
        totplnk_delta = totplnk_delta,
        totplnk = totplnk,
        gpoint_flavor = gpoint_flavor,
        sfc_src = sfc_src,
        lay_src = lay_src,
        lev_src = lev_src,
        sfc_src_jac = sfc_src_jac
)

rrtmgp_compute_Planck_source(*list(planck_kwargs.values()))

In [ ]:
gas_names
gas_minor = [g.strip().decode() for g in kdist["gas_minor"].values]
minor_gases_lower_red




#   subroutine create_idx_minor(gas_names, &
#     gas_minor, identifier_minor, minor_gases_atm, idx_minor_atm)
#     character(len=*), dimension(:), intent(in) :: gas_names
#     character(len=*), dimension(:), intent(in) :: &
#                                                   gas_minor, &
#                                                   identifier_minor
#     character(len=*), dimension(:), intent(in) :: minor_gases_atm
#     integer, dimension(:), allocatable, &
#                                    intent(out) :: idx_minor_atm

#     ! local
#     integer :: imnr
#     integer :: idx_mnr
#     allocate(idx_minor_atm(size(minor_gases_atm,dim=1)))
#     do imnr = 1, size(minor_gases_atm,dim=1) ! loop over minor absorbers in each band
#       ! Find identifying string for minor species in list of possible identifiers (e.g. h2o_slf)
#       idx_mnr     = string_loc_in_array(minor_gases_atm(imnr), identifier_minor)
#       ! Find name of gas associated with minor species identifier (e.g. h2o)
#       idx_minor_atm(imnr) = string_loc_in_array(gas_minor(idx_mnr),    gas_names)
#     enddo
#   end subroutine create_idx_minor

In [6]:
from pyrte_rrtmgp.pyrte_rrtmgp import rrtmgp_compute_tau_absorption

nminorlower = len(kdist["minor_absorber_intervals_lower"])
nminorklower = len(kdist["contributors_lower"])
nminorupper = len(kdist["minor_absorber_intervals_upper"])
nminorkupper = len(kdist["contributors_upper"])

idx_h2o = list(gas_names).index("h2o") + 1

tau_kwargs = dict(
    ncol = ncol,
    nlay = nlay,
    nband = nbnd,
    ngpt = ngpt,
    ngas = len(gas_names),
    nflav = nflav,
    neta = neta,
    npres = npres,
    ntemp = ntemp,
    nminorlower = nminorlower,
    nminorklower = nminorklower,
    nminorupper = nminorupper,
    nminorkupper = nminorkupper,
    idx_h2o = idx_h2o,
    gpoint_flavor = gpoint_flavor,
    band_lims_gpt = band_lims_gpt,
    kmajor = kdist["kmajor"].values,
    kminor_lower = kdist["kminor_lower"].values,
    kminor_upper = kdist["kminor_upper"].values,
    minor_limits_gpt_lower = kdist["minor_limits_gpt_lower"].values.T,
    minor_limits_gpt_upper = kdist["minor_limits_gpt_upper"].values.T,
    minor_scales_with_density_lower = kdist["minor_scales_with_density_lower"].values,
    minor_scales_with_density_upper = kdist["minor_scales_with_density_upper"].values,
    scale_by_complement_lower = kdist["scale_by_complement_lower"].values,
    scale_by_complement_upper = kdist["scale_by_complement_upper"].values,


    # ! Get index of gas (if present) for determining col_gas
    # call create_idx_minor(this%gas_names, gas_minor, identifier_minor, minor_gases_lower_red, this%idx_minor_lower)
    # call create_idx_minor(this%gas_names, gas_minor, identifier_minor, minor_gases_upper_red, this%idx_minor_upper)
    # ! Get index of gas (if present) that has special treatment in density scaling
    # call create_idx_minor_scaling(this%gas_names, scaling_gas_lower_red, this%idx_minor_scaling_lower)
    # call create_idx_minor_scaling(this%gas_names, scaling_gas_upper_red, this%idx_minor_scaling_upper)


    idx_minor_lower = idx_minor_lower,
    idx_minor_upper = idx_minor_upper,
    idx_minor_scaling_lower = idx_minor_scaling_lower,
    idx_minor_scaling_upper = idx_minor_scaling_upper,
    kminor_start_lower = kminor_start_lower,
    kminor_start_upper = kminor_start_upper,
    tropo = tropo!=0,
    col_mix = col_mix,
    fmajor = fmajor,
    fminor = fminor,
    play = play,
    tlay = tlay,
    col_gas = col_gas,
    jeta = jeta,
    jtemp = jtemp,
    jpress = jpress,
    tau = np.ndarray([ncol, nlay, nbnd], dtype=np.float64)
)

rrtmgp_compute_tau_absorption(*list(tau_kwargs.values()))

KeyError: "No variable named 'idx_minor_lower'. Variables on the dataset include ['key_species', 'kmajor', 'plank_fraction', 'temperature_Planck', 'totplnk', ..., 'kminor_lower', 'kminor_start_lower', 'kminor_upper', 'kminor_start_upper', 'optimal_angle_fit']"

In [49]:
# default values
n_quad_angs = 1

max_gauss_pts = 4
# Assuming max_gauss_pts is defined
gauss_Ds = np.array([1.66, 0., 0., 0.,  # Diffusivity angle, not Gaussian angle
                     1.18350343, 2.81649655, 0., 0.,
                     1.09719858, 1.69338507, 4.70941630, 0.,
                     1.06056257, 1.38282560, 2.40148179, 7.15513024])

# Reshape gauss_Ds
gauss_Ds = gauss_Ds.reshape((max_gauss_pts, max_gauss_pts))

gauss_wts = np.array([0.5, 0., 0., 0.,
                      0.3180413817, 0.1819586183, 0., 0.,
                      0.2009319137, 0.2292411064, 0.0698269799, 0.,
                      0.1355069134, 0.2034645680, 0.1298475476, 0.0311809710])

# Reshape gauss_wts
gauss_wts = gauss_wts.reshape((max_gauss_pts, max_gauss_pts))

secants = np.empty((ncol, ngpt, n_quad_angs))
# Assuming ngpt, ncol, and secants are defined
for imu in range(n_quad_angs):
    for igpt in range(ngpt):
        for icol in range(ncol):
            secants[icol, igpt, imu] = gauss_Ds[imu, n_quad_angs]

weights = gauss_wts[1:n_quad_angs,n_quad_angs]

rte_lw_solver_noscat_kwargs = dict(
    ncol = ncol,
    nlay = nlay,
    ngpt = ngpt,
    top_at_1 = top_at_1,
    nmus = n_quad_angs,
    Ds = secants,
    weights = weights,
    tau = rfmip["optical_thickness"].values,
    lay_source = lay_src,
    lev_source = lev_src,
    sfc_emis = sfc_emis,
    sfc_src = sfc_src,
    inc_flux = flxdn_file["rsd"].values,
    flux_up = flxup_file["rsu"].values,
    flux_dn = np.zeros((ncol, ngpt)),
    do_broadband = False,
    broadband_up = np.zeros((ncol, ngpt)),
    broadband_dn = np.zeros((ncol, ngpt)),
    do_Jacobians = False,
    sfc_srcJac = sfc_src_jac,
    flux_upJac = np.zeros((ncol, ngpt)),
    do_rescaling = False,
    ssa = np.zeros((ncol, ngpt)),
    g = np.zeros((ncol, ngpt))
)


SyntaxError: invalid syntax (637972954.py, line 1)

In [ ]:
plev = rfmip["pres_level"]

# get_col_dry(vmr(:,:,idx_h2o), plev)

#   function get_col_dry(vmr_h2o, plev, latitude) result(col_dry)
#     ! input
#     real(wp), dimension(:,:), intent(in) :: vmr_h2o  ! volume mixing ratio of water vapor to dry air; (ncol,nlay)
#     real(wp), dimension(:,:), intent(in) :: plev     ! Layer boundary pressures [Pa] (ncol,nlay+1)
#     real(wp), dimension(:),   optional, &
#                               intent(in) :: latitude ! Latitude [degrees] (ncol)
#     ! output
#     real(wp), dimension(size(plev,dim=1),size(plev,dim=2)-1) :: col_dry ! Column dry amount (ncol,nlay)
#     ! ------------------------------------------------
#     ! first and second term of Helmert formula
#     real(wp), parameter :: helmert1 = 9.80665_wp
#     real(wp), parameter :: helmert2 = 0.02586_wp
#     ! local variables
#     real(wp), dimension(size(plev,dim=1)) :: g0 ! (ncol)
#     real(wp):: delta_plev, m_air, fact
#     integer :: ncol, nlev
#     integer :: icol, ilev ! nlay = nlev-1
#     ! ------------------------------------------------
#     ncol = size(plev, dim=1)
#     nlev = size(plev, dim=2)
#     !$acc        data    create(g0)
#     !$omp target data map(alloc:g0)
#     if(present(latitude)) then
#       ! A purely OpenACC implementation would probably compute g0 within the kernel below
#       !$acc parallel loop
#       !$omp target teams distribute parallel do simd
#       do icol = 1, ncol
#         g0(icol) = helmert1 - helmert2 * cos(2.0_wp * pi * latitude(icol) / 180.0_wp) ! acceleration due to gravity [m/s^2]
#       end do
#     else
#       !$acc parallel loop
#       !$omp target teams distribute parallel do simd
#       do icol = 1, ncol
#         g0(icol) = grav
#       end do
#     end if

#     !$acc                parallel loop gang vector collapse(2) copyin(plev,vmr_h2o)  copyout(col_dry)
#     !$omp target teams distribute parallel do simd collapse(2) map(to:plev,vmr_h2o) map(from:col_dry)
#     do ilev = 1, nlev-1
#       do icol = 1, ncol
#         delta_plev = abs(plev(icol,ilev) - plev(icol,ilev+1))
#         ! Get average mass of moist air per mole of moist air
#         fact = 1._wp / (1.+vmr_h2o(icol,ilev))
#         m_air = (m_dry + m_h2o * vmr_h2o(icol,ilev)) * fact
#         col_dry(icol,ilev) = 10._wp * delta_plev * avogad * fact/(1000._wp*m_air*100._wp*g0(icol))
#       end do
#     end do
#     !$acc end        data
#     !$omp end target data
#   end function get_col_dry

In [ ]:
kdist

ncol = len(rfmip["site"])
nlay = len(rfmip["layer"])
ndbnd = kdist.dims["bnd"]
ngpt = kdist.dims["gpt"] # int(read_field(ncid, 'bnd_limits_gpt', 2, nbnds))
nflav = kdist.dims["pair"]
# Table kmajor has dimensions (ngpt, neta, npres, ntemp)
neta = kdist.dims["mixing_fraction"]
npres = kdist.dims["pressure"]
ntemp = kdist.dims["temperature"]
nPlanckTemp = kdist.dims["temperature_Planck"]
tlay = rfmip["temp_layer"]
tlev = rfmip["temp_level"]
play = rfmip["pres_layer"]
plev = rfmip["pres_level"]
tsfc = rfmip["surface_temperature"]
sfc_lay = nlay if top_at_1 else 1 # merge(nlay, 1, top_at_1)
# fmajor = 

# py::array_t<double> fmajor,
# py::array_t<int> jeta,
# py::array_t<bool> tropo,
# py::array_t<int> jtemp,
# py::array_t<int> jpress,
# py::array_t<int> gpoint_bands,
# py::array_t<int> band_lims_gpt,
# py::array_t<double> pfracin,
# double temp_ref_min,
# double totplnk_delta,
# py::array_t<double> totplnk,
# py::array_t<int> gpoint_flavor,
# py::array_t<double> sfc_src,
# py::array_t<double> lay_src,
# py::array_t<double> lev_src,
# py::array_t<double> sfc_src_jac

# compute_Planck_source(ncol, nlay, nbnd, ngpt, &
#                 get_nflav(this), this%get_neta(), this%get_npres(), this%get_ntemp(), this%get_nPlanckTemp(), &
#                 tlay, tlev_wk, tsfc, merge(nlay, 1, top_at_1), &
#                 fmajor, jeta, tropo, jtemp, jpress,                    &
#                 this%get_gpoint_bands(), this%get_band_lims_gpoint(), this%planck_frac, this%temp_ref_min,&
#                 this%totplnk_delta, this%totplnk, this%gpoint_flavor,  &
#                 sources%sfc_source, sources%lay_source, sources%lev_source, &
#                 sources%sfc_source_Jac)

# rrtmgp_compute_Planck_source(
#         int ncol,
#         int nlay,
#         int nbnd,
#         int ngpt,
#         int nflav,
#         int neta,
#         int npres,
#         int ntemp,
#         int nPlanckTemp,
#         py::array_t<double> tlay,
#         py::array_t<double> tlev,
#         py::array_t<double> tsfc,
#         int sfc_lay,
#         py::array_t<double> fmajor,
#         py::array_t<int> jeta,
#         py::array_t<bool> tropo,
#         py::array_t<int> jtemp,
#         py::array_t<int> jpress,
#         py::array_t<int> gpoint_bands,
#         py::array_t<int> band_lims_gpt,
#         py::array_t<double> pfracin,
#         double temp_ref_min,
#         double totplnk_delta,
#         py::array_t<double> totplnk,
#         py::array_t<int> gpoint_flavor,
#         py::array_t<double> sfc_src,
#         py::array_t<double> lay_src,
#         py::array_t<double> lev_src,
#         py::array_t<double> sfc_src_jac
#     )


# rrtmgp_compute_Planck_source


# optical_props = 

# load rrtmgp_rfmip_lw.F90 load_and_init ref to datasets used
rfmip



rfmip["pres_layer"]
rfmip["pres_level"]
rfmip["temp_layer"]
rfmip["temp_level"]
rfmip["surface_temperature"]



from pyrte.pyrte import rte_lw_solver_noscat



# Set of optical properties as one or more arrays
# type, public :: ty_optical_props
# integer,  dimension(:,:), allocatable, private :: band2gpt      ! (begin g-point, end g-point) = band2gpt(2,band)
# integer,  dimension(:),   allocatable, private :: gpt2band      ! band = gpt2band(g-point)
# real(wp), dimension(:,:), allocatable, private :: band_lims_wvn ! (upper and lower wavenumber by band) = band_lims_wvn(2,band)
# character(len=name_len),               private :: name = ""
optical_props = rfmip["optical_thickness"]

# Is the top of the domain at index 1? (if not, ordering is bottom-to-top)
top_at_1 = True

# Derived type with Planck source functions
# type, extends(ty_optical_props), public :: ty_source_func_lw
# real(wp), allocatable, dimension(:,:,:) :: lay_source
#     !! Planck source at layer average temperature (ncol, nlay, ngpt)
# real(wp), allocatable, dimension(:,:,:) :: lev_source
#     !! Planck source at layer edge (ncol, nlay+1, ngpt)
# real(wp), allocatable, dimension(:,:  ) :: sfc_source
#     !! Planck function at surface temperature
# real(wp), allocatable, dimension(:,:  ) :: sfc_source_Jac
#     !! surface source Jacobian
sources = []

# emissivity at surface [] (nband, ncol)
sfc_emis = []

# Dervied type for computing spectral integrals from g-point fluxes.
# Default computes broadband fluxes at all levels if output arrays are defined. Can be extended per user desires.
# type, abstract, public :: ty_fluxes
# contains
# procedure(reduce_abstract),      deferred, public :: reduce
# procedure(are_desired_abstract), deferred, public :: are_desired
# end type ty_fluxes
fluxes = [] 

# OPTIONALS
# Number of angles used in Gaussian quadrature (max 3, no-scattering solution)
n_gauss_angles = 1
# When 2-stream parameters (tau/ssa/g) are provided, use 2-stream methods
# Default is to use re-scaled longwave transport
use_2stream = False
# User-specifed 1/cos of transport angle per col, g-point
lw_Ds = []

# TARGETS
# incident flux at domain top [W/m2] (ncol, ngpts)
inc_flux = []
# surface temperature flux  Jacobian [W/m2/K] (ncol, nlay+1)
flux_up_Jac = []





# call lw_solver_noscat(ncol, nlay, ngpt,                 &
#                     logical(top_at_1, wl), n_quad_angs,         &
#                     secants, gauss_wts(1:n_quad_angs,n_quad_angs), &
#                     optical_props%tau,                 &
#                     sources%lay_source,                &
#                     sources%lev_source,                &
#                     sfc_emis_gpt, sources%sfc_source,  &
#                     inc_flux_diffuse,                  &
#                     gpt_flux_up, gpt_flux_dn,          &
#                     do_broadband, flux_up_loc, flux_dn_loc,     &
#                     logical(do_Jacobians, wl), sources%sfc_source_Jac, jacobian, &
#                     logical(.false., wl),  optical_props%tau, optical_props%tau)
#                                             ! The last two arguments won't be used since the
#                                             ! third-to-last is .false. but need valid addresses




result = rte_lw_solver_noscat(
)